# Convolutional Neural Network

### Importing the libraries

In [22]:
!pip install --upgrade tensorflow keras Pillow

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


In [78]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [79]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [80]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [81]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [95]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [96]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [97]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [98]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [99]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [100]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [101]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [102]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [103]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [104]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 35s 139ms/step - loss: 0.6924 - accuracy: 0.5273 - val_loss: 0.6631 - val_accuracy: 0.6120
Epoch 2/25
250/250 [==============================] - 35s 138ms/step - loss: 0.6402 - accuracy: 0.6297 - val_loss: 0.6034 - val_accuracy: 0.6735
Epoch 3/25
250/250 [==============================] - 34s 137ms/step - loss: 0.5742 - accuracy: 0.7067 - val_loss: 0.5310 - val_accuracy: 0.7395
Epoch 4/25
250/250 [==============================] - 35s 139ms/step - loss: 0.5228 - accuracy: 0.7428 - val_loss: 0.4925 - val_accuracy: 0.7695
Epoch 5/25
250/250 [==============================] - 34s 137ms/step - loss: 0.4882 - accuracy: 0.7644 - val_loss: 0.4518 - val_accuracy: 0.7875
Epoch 6/25
250/250 [==============================] - 35s 138ms/step - loss: 0.4662 - accuracy: 0.7790 - val_loss: 0.4632 - val_accuracy: 0.7875
Epoch 7/25
250/250 [==============================] - 34s 136ms/step - loss: 0.4356 - accuracy: 0.7927 - val_loss: 0.4526 - val_ac

## Part 4 - Making a single prediction

In [93]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [94]:
print(prediction)

dog


## Fim